## Dates, times, and datetimes

One of the most common data types you will be interacting with as a data engineer is temporal data. This can range from timestamps showing when records were created, to a date telling when a book was published, to a range of time for which a process was running. Because of the prevalence of temporal data, Python has an entire library to handle date and time data, called `datetime`. To import a python library, we just need to do the following:

In [2]:
import datetime

Using this library, let's create our first `datetime` object using the `datetime.datetime.now()` method:

In [3]:
current_time = datetime.datetime.now()
print(current_time)
print(type(current_time))

2021-04-23 09:53:36.492874
<class 'datetime.datetime'>


As we can see from the print statements, this creates a datetime class object with the time in the current timezone. This illustrates one of the first issues with a `datetime`: timezones. Python tends to default to the local timezone in functions, but one must always check the documentation or specify a `datetime.tzone` object to verify behavior; awareness of timezone is the first step to not being tripped up when times do not line up as expected. One important timezone for us to consider is Universal Coordinated Time or UTC. This is the standard reference timezone from which all others can be calculated as a difference. If we would like the current time in UTC, we can use the `utcnow` method of `datetime`:

In [4]:
current_time = datetime.datetime.now()
utc_current_time = datetime.datetime.utcnow()
print(f"Local time is{current_time}")
print(f"But UTC time is {utc_current_time}")

Local time is2021-04-23 10:01:29.487813
But UTC time is 2021-04-23 17:01:29.487848


If we would like to create a `datetime` object for a specific time, we can use the `datetime.datetime()` method:

In [5]:
new_years = datetime.datetime(2021, 1, 1)
print(new_years)

2021-01-01 00:00:00


`datetime.datetime` also has argements for hour, minute, second, microsecond, and `tzone`. Let's create a `datetime` for 5 minute after midnight on January 1st using the kwarg `minute`:

In [8]:
new_years_and_five = datetime.datetime(2021, 1, 1, minute=5)
print(new_years_and_five)

2021-01-01 00:05:00


Until now we've been printing the full datetime object. But we can also print specific fields which are stored as attributes of the class:

In [11]:
current_time = datetime.datetime.now()
print(current_time.day)
print(current_time.second)
print(current_time.tzinfo)

23
5
None


In addition to printing the specific attributes of a `datetime`, we often want to create a specific `str` representation of the time. To do this we use the `strftime()` method of a `datetime` object. This method takes a single argument, a formatting `str` with the desired output format for the date `str`. A handy reference guide for the format can be [found here](https://strftime.org/). For example, if we want to return a string with the format `month_name day, full_year` we would use the formatting `str`: `"%B %d, %Y"`:

In [12]:
print(current_time.strftime("%B %d, %Y"))

April 23, 2021


We can also use these formatting strings to create `datetime` objects from `str` input, a very common data engineering task. To do this, we need an input `str` and a formatting `str` telling us how to interpret the input. Let's read in a date in the same format that we just printed:

In [16]:
date_format =  "%B %d, %Y"
first_of_may = datetime.datetime.strptime("May 1, 2020", date_format)
print(first_of_may, type(first_of_may))

# and now let's try a different format:
new_format = "%Y-%m-%d %H:%M:%S"
feb_29 = datetime.datetime.strptime("2020-02-29 10:04:31", new_format)
print(feb_29)


2020-05-01 00:00:00 <class 'datetime.datetime'>
2020-02-29 10:04:31


#### Time and Date
In additon to the combined datetime class we've been exploring, `datetime` also has separate `time` and `date` classes which contain just the specified portions of temporal data. 

### Time deltas
Dates and times are great for capturing when events occured, but we often also want to know how much time passes between events. This is where the `timedelta` object comes in. The quickest was to see how these work is simply use the `-` operator between two `datetime` objects:

In [18]:
jan_first = datetime.datetime(2021, 1, 1)
year_so_far = datetime.datetime.now() - jan_first
print(year_so_far)
print(type(year_so_far))

116 days, 13:27:25.031352
<class 'datetime.timedelta'>


We can also create `timedelta` objects directly using the `datetime.timedelta` class constructor. Let's make a timedelta of 1 day and then add it to `jan_first` and see how it behaves:

In [21]:
day_delta = datetime.timedelta(days=1)
print(day_delta, type(day_delta))
jan_two = jan_first + day_delta
print(jan_two, type(jan_two))
print(jan_two + day_delta)

1 day, 0:00:00 <class 'datetime.timedelta'>
2021-01-02 00:00:00 <class 'datetime.datetime'>
2021-01-03 00:00:00


As you can see, adding a `timedelta` to a `datetime` results in another `datetime` object. This behavior makes it straightforward to use `timedelta`s to increment `datetime`s.

### Book datetime
Let's put this new knowledge to use by updating our `Book` class. We can now add a published year to our `__init__()` function that can accept either a `str`, an `int`, or a `datetime` object:

In [34]:
class Book:
    def __init__(self, title, author, genre, pub_year=None):
        self.title = title
        self.author = author
        self.genre = genre
        if isinstance(pub_year, datetime.datetime):
            self.pub_year = pub_year
        elif isinstance(pub_year, str):
            self.pub_year = datetime.datetime.strptime(pub_year, "%Y")
        elif isinstance(pub_year, int):
            self.pub_year = datetime.datetime(year=pub_year, month=1, day=1)
        else:
            self.pub_year = pub_year
    
    def have_you_read(self):
        print(f"Have you read {self.title} by {self.author}?")
    
    def was_published(self):
        print(f"{self.title} was published in {self.pub_year.year}")

We can also use our knowledge of `datetime` objects to create a subclass of `Book`, `LibraryBook` that has an attribute for when it is checked out, its due date (default loan length of 30 days), and a function that returns how long the book has been checked out for:

In [35]:
class LibraryBook(Book):
    def __init__(self, title, author, genre, pub_year=None, checked_out=None, loan_length=None):
        super().__init__(title, author, genre, pub_year)
        if isinstance(checked_out, datetime.datetime):
            self.checked_out = checked_out
        else:
            self.checked_out = datetime.datetime.now()
        if isinstance(loan_length, datetime.timedelta):
            self.due_date = self.checked_out + loan_length
        else:
            self.due_date = self.checked_out + datetime.timedelta(days=30)
    def on_loan(self):
        cur_dur = datetime.datetime.now() - self.checked_out
        print(f"{self.title} has been on loan for {cur_dur}")

In [36]:
from time import sleep
runaway_ralph = LibraryBook("Runaway Ralph", "Beverly Cleary", "Children's Novel", 1970)
runaway_ralph.have_you_read()
runaway_ralph.was_published()
runaway_ralph.on_loan()
sleep(5)
runaway_ralph.on_loan()

Have you read Runaway Ralph by Beverly Cleary?
Runaway Ralph was published in 1970
Runaway Ralph has been on loan for 0:00:00.000513
Runaway Ralph has been on loan for 0:00:05.003152


Sometimes it is useful to convert a `datetime` into a number of seconds. Here's an example of how to do that. First we initialize a string containing only a time (not a date). When we convert to a datetime representation, a default date (1/1/1900) gets added automatically. To convert the original time to a number of seconds, we have to remove the date portion from `date_time`. Then, we can call `total_seconds()` on the resulting `timedelta` to get a number of seconds represented by our original time string:

In [3]:
# initialize a time string
time_string = "12:01:27"

# convert to datetime representation
date_time = datetime.datetime.strptime(time_string, "%H:%M:%S")
print(date_time)

#remove years, months, and days so only time (not date) is left
a_timedelta = date_time - datetime.datetime(1900, 1, 1)
seconds = a_timedelta.total_seconds()
print(seconds)

1900-01-01 12:01:27
43287.0


### Further Reading
- [Python datetime docs](https://docs.python.org/3/library/datetime.html)